In [1]:
!git clone https://github.com/facebookresearch/dlrm.git

fatal: destination path 'dlrm' already exists and is not an empty directory.


In [2]:
# experiment params
num_train_trials = 5
num_train_warmups = 1
num_jit_trials = 10
num_jit_warmups = 2
num_inference_trials = 10000
num_inference_warmups = 1000

In [3]:
import sys
from pathlib import Path

sys.path.append(str(Path.cwd()/Path("dlrm")))
print(sys.path)

from dlrm.dlrm_s_pytorch import *

['/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/achilibe/Code/2JITsu/venv/lib/python3.10/site-packages', '/home/achilibe/Code/2JITsu/ViT/torch/vit_pytorch', '/home/achilibe/Code/2JITsu/BERT/torch/benchmark/torchbenchmark/models/BERT_pytorch', '/home/achilibe/Code/2JITsu/DLRM/torch/dlrm']
Unable to import mlperf_logging,  No module named 'mlperf_logging'


2024-10-18 13:22:52.704851: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-18 13:22:52.712210: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-18 13:22:52.720291: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-18 13:22:52.722723: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-18 13:22:52.729327: I tensorflow/core/platform/cpu_feature_guar

In [4]:
import tqdm
import time
import torch

dlrm = DLRM_Net(
    16,
    np.array([4,3,2]),
    np.array([13,512,256,64,16]),
    np.array([512,256,1]),
    arch_interaction_op='cat',
    arch_interaction_itself=False,
    sigmoid_bot=-1,
    sigmoid_top=2
)

def time_jit(num_jit_runs: int, num_jit_warmups: int, model: nn.Module) -> float:
    times = []
    for i in tqdm.trange(1, num_jit_runs + 1):
        start_jit_time = time.time()
        
        jit_model = torch.compile(
            model, 
            options={"triton.cudagraphs": True}, 
            fullgraph=True,
            backend='cudagraphs'
        )

        end_jit_time = time.time()
        torch.compiler.reset()
        if i >= num_jit_warmups:
            times.append(end_jit_time - start_jit_time)

    return sum(times) / len(times)
    
print(torch._dynamo.list_backends())
average_jit_time = time_jit(num_jit_trials, num_jit_warmups, dlrm)
print(f"Average JIT time: {average_jit_time}")

['cudagraphs', 'inductor', 'onnxrt', 'openxla', 'tvm']


100%|████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 21.19it/s]

Average JIT time: 0.00011865297953287761


In [5]:
def train


for j, inputBatch in enumerate(train_ld):
    X, lS_o, lS_i, T, W, CBPP = unpack_batch(inputBatch)

    # early exit if nbatches was set by the user and has been exceeded
    if nbatches > 0 and j >= nbatches:
        break

    # Skip the batch if batch size not multiple of total ranks
    if ext_dist.my_size > 1 and X.size(0) % ext_dist.my_size != 0:
        print(
            "Warning: Skiping the batch %d with size %d"
            % (j, X.size(0))
        )
        continue

    mbs = T.shape[0]  # = args.mini_batch_size except maybe for last

    start_time = time.time()
    # forward pass
    Z = dlrm_wrap(
        X,
        lS_o,
        lS_i,
        use_gpu,
        device,
        ndevices=ndevices,
    )

    if ext_dist.my_size > 1:
        T = T[ext_dist.get_my_slice(mbs)]
        W = W[ext_dist.get_my_slice(mbs)]

    # loss
    E = loss_fn_wrap(Z, T, use_gpu, device)

    # compute loss and accuracy
    L = E.detach().cpu().numpy()  # numpy array
    # training accuracy is not disabled
    # S = Z.detach().cpu().numpy()  # numpy array
    # T = T.detach().cpu().numpy()  # numpy array

    # # print("res: ", S)

    # # print("j, train: BCE ", j, L)

    # mbs = T.shape[0]  # = args.mini_batch_size except maybe for last
    # A = np.sum((np.round(S, 0) == T).astype(np.uint8))
    E.backward()

    # optimizer
    optimizer.step()
    lr_scheduler.step()
    end_time = time.time()


    model_metrics_dict, is_best = inference(
        args,
        dlrm,
        best_acc_test,
        best_auc_test,
        test_ld,
        device,
        use_gpu,
        log_iter,
    )



SyntaxError: invalid syntax (626569888.py, line 1)